# 10장 튜닝 기법

* "부록3 매트플롯립 입문"에서 한글 폰트를 올바르게 출력하기 위한 설치 방법을 설명했다. 설치 방법은 다음과 같다.

In [ ]:
!sudo apt-get install -y fonts-nanum* | tail -n 1
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib

In [ ]:
# 필요 라이브러리 설치

!pip install torchviz | tail -n 1
!pip install torchinfo | tail -n 1

* 모든 설치가 끝나면 한글 폰트를 바르게 출력하기 위해 **[런타임]** -> **[런타임 다시시작]**을 클릭한 다음, 아래 셀부터 코드를 실행해 주십시오.

In [ ]:
# 라이브러리 임포트

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# 폰트 관련 용도
import matplotlib.font_manager as fm

# 나눔 고딕 폰트의 경로 명시
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=path, size=10).get_name()

In [ ]:
# 파이토치 관련 라이브러리

import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchviz import make_dot
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets

In [ ]:
# warning 표시 끄기
import warnings
warnings.simplefilter('ignore')

# 기본 폰트 설정
plt.rcParams['font.family'] = font_name

# 기본 폰트 사이즈 변경
plt.rcParams['font.size'] = 14

# 기본 그래프 사이즈 변경
plt.rcParams['figure.figsize'] = (6,6)

# 기본 그리드 표시
# 필요에 따라 설정할 때는, plt.grid()
plt.rcParams['axes.grid'] = True

# 마이너스 기호 정상 출력
plt.rcParams['axes.unicode_minus'] = False

# 넘파이 부동소수점 자릿수 표시
np.set_printoptions(suppress=True, precision=4)

In [ ]:
# GPU 디바이스 할당

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# 분류 클래스 명칭 리스트
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# 분류 클래스 수,　10
n_output = len(list(set(classes)))

# 결과 확인
print(n_output)

## 10.3 최적화 함수

### SGD

In [ ]:
# 가상의 파라미터와 그래디언트 생성
W = torch.randn(3, 3, requires_grad=True)
B = torch.randn(3, requires_grad=True)
W.grad = torch.randn(3, 3)
B.grad = torch.randn(3)

# 파라미터 갱신
# 학습률
lr = 0.001
# 경사를 기반으로 파라미터 갱신
W.data -= lr * W.grad.data
B.data -= lr * B.grad.data

print("Updated W:\n", W)
print("Updated B:\n", B)

### 모멘텀

In [ ]:
# 가상의 모델 파라미터
net_params = [torch.randn(10, 1, requires_grad=True)]

# 모멘텀
optimizer = optim.SGD(net_params, lr=lr, momentum=0.9) # momentum 값을 0.9로 지정

print(optimizer)

In [ ]:
net_params

### Adam

In [ ]:
# 가상의 모델 파라미터
net_params = [torch.randn(10, 1, requires_grad=True)]

# Adam
optimizer = optim.Adam(net_params)

print(optimizer)

## 10.4 과학습의 대응 방법

### 드랍 아웃 함수 동작 확인

In [ ]:
# 드랍 아웃 실험용 더미 데이터 작성
torch.manual_seed(123) # 결과를 재현하기 위해 난수 시드 고정
inputs = torch.randn(1, 10) # 1x10 크기의 텐서 생성
print("Original Inputs:")
print(inputs)

In [ ]:
# 드랍 아웃 함수 정의
dropout = nn.Dropout(0.5) # 50%의 확률로 뉴런을 비활성화

# 훈련 페이즈에서의 거동
dropout.train() # 모델을 훈련 모드로 설정
print(f"Is in training mode? : {dropout.training}")
outputs = dropout(inputs)
print("Outputs in Train Mode:")
print(outputs)

# 예측 페이즈에서의 거동
dropout.eval() # 모델을 평가 모드로 설정
print(f"Is in training mode? : {dropout.training}")
outputs = dropout(inputs)
print("Outputs in Eval Mode:")
print(outputs)

### 데이터 증강

In [ ]:
# Transforms를 사용한 데이터 증강 구현
# 훈련 데이터용: 정규화에 추가로 반전과 RandomErasing 수행
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5), # 50% 확률로 좌우 반전
    transforms.ToTensor(), # 이미지를 텐서로 변환
    # 이미지 [0-255] px 단위 범위 : [255,0,0] >> 텐서 [0-1] 실수 [1,0,0]
    transforms.Normalize((0.5,), (0.5,)), # 정규화(평균, 표준편차) mean: 0.5, std: 0.5
    #  x - mean / sigma(std) = (x-0.5) / 0.5
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)
    # 50% 확률로 이미지의 일부 영역을 지움
    # scale = 0.02(2%)~0.33(33%) 영역을 지운다.영역을 가린다
    # ratio=(0.3, 3.3): 가려진 영역 가로/세로 비율
    # value=0 : 검정색(0) 채움
])

print(transform_train)

## 10.5 공통 함수의 라이브러리화

In [ ]:
# 공통 함수 다운로드
!git clone https://github.com/wikibook/pythonlibs.git

# 공통 함수 불러오기
from pythonlibs.torch_lib1 import *

# 공통 함수 확인
print(README)

## 데이터 준비

In [ ]:
# Transforms의 정의

transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize(0.5, 0.5) #(평균 mean, 표준편차 std)
])
# 텐서 변환 >> 정규화 해줘

In [ ]:
# 데이터 취득용 함수 dataset

data_root = './data'

train_set = datasets.CIFAR10(
    root = data_root, train = True,
    download = True, transform = transform)

# 검증 데이터셋
test_set = datasets.CIFAR10(
    root = data_root, train = False,
    download = True, transform = transform)

In [ ]:
# 미니 배치 사이즈 지정
batch_size = 100

# 훈련용 데이터로더
# 훈련용이므로 셔플을 True로 설정
train_loader = DataLoader(train_set,
    batch_size = batch_size, shuffle = True)

# 검증용 데이터로더
# 검증용이므로 셔플하지 않음
test_loader = DataLoader(test_set,
    batch_size = batch_size, shuffle = False)

In [ ]:
# 처음 50개 이미지 출력
show_images_labels(test_loader, classes, None, None)
# None, None : 모델, 디바이스

## 10.6 층을 깊게 쌓은 모델 구현하기

In [ ]:
class CNN_v2(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # 합성곱, ReLU, MaxPool 레이어 정의
        self.conv1 = nn.Conv2d(3, 32, 3, padding=(1,1))
        # padding=(1,1) 이미지 크기 유지
        self.conv2 = nn.Conv2d(32, 32, 3, padding=(1,1))
        self.conv3 = nn.Conv2d(32, 64, 3, padding=(1,1))
        self.conv4 = nn.Conv2d(64, 64, 3, padding=(1,1))
        self.conv5 = nn.Conv2d(64, 128, 3, padding=(1,1))
        self.conv6 = nn.Conv2d(128, 128, 3, padding=(1,1))
        self.relu = nn.ReLU(inplace=True)
        self.flatten = nn.Flatten()
        self.maxpool = nn.MaxPool2d((2,2))

        # 선형(Fully Connected) 레이어 정의
        self.l1 = nn.Linear(4*4*128, 128)
        self.l2 = nn.Linear(128, num_classes)

        # 레이어들을 순차적으로 실행할 `nn.Sequential` 정의
        self.features = nn.Sequential(
            self.conv1,
            self.relu,
            self.conv2,
            self.relu,
            self.maxpool,
            self.conv3,
            self.relu,
            self.conv4,
            self.relu,
            self.maxpool,
            self.conv5,
            self.relu,
            self.conv6,
            self.relu,
            self.maxpool,
            )

        self.classifier = nn.Sequential(
            self.l1,
            self.relu,
            self.l2
        )

    def forward(self, x):
        x1 = self.features(x)
        x2 = self.flatten(x1)
        x3 = self.classifier(x2)
        return x3

In [ ]:
# 손실 계산 그래프 시각화
net = CNN_v2(n_output).to(device)
criterion = nn.CrossEntropyLoss()
loss = eval_loss(test_loader, device, net, criterion)
g = make_dot(loss, params=dict(net.named_parameters()))
display(g)

In [ ]:
# 난수 고정
torch_seed()

# 모델 인스턴스 생성
lr = 0.01
net = CNN_v2(n_output).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr)
history = np.zeros((0, 5))

In [ ]:
# 학습

num_epochs = 50
history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history)

In [ ]:
evaluate_history(history)

## 10.7 최적화 함수 선택

### 모멘텀 설정

In [ ]:
# 난수 고정
torch_seed()

# 모델 인스턴스 생성
lr = 0.01
net = CNN_v2(n_output).to(device)
criterion = nn.CrossEntropyLoss()

# 최적화 함수에 모멘텀 값 설정
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
# momentum=0.9 : 이전 업데이트되는 방향을 90% 반영 >> 학습속도 높이고 안정화
history2 = np.zeros((0, 5))

In [ ]:
# 학습

num_epochs = 20
history2 = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history2)

In [ ]:
evaluate_history(history2)

### Adam 함수 사용

In [ ]:
# 난수 고정
torch_seed()

# 모델 인스턴스 생성
net = CNN_v2(n_output).to(device)
criterion = nn.CrossEntropyLoss()

# 최적화 함수를 Adam으로 교체
optimizer = optim.Adam(net.parameters())
history3 = np.zeros((0, 5))

In [ ]:
print(optimizer)

In [ ]:
# 학습

num_epochs = 20
history3 = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history3)

In [ ]:
evaluate_history(history3)

### 결과 비교

In [ ]:
# 결과 비교(검증 데이터의 정확도)
plt.figure(figsize=(9,8))
plt.plot(history[:,0], history[:,4], label='SGD', c='k',ls='dashed' )
plt.plot(history2[:,0], history2[:,4], label='SGD momentum=0.9', c='k')
plt.plot(history3[:,0], history3[:,4], label='Adam', c='b')
plt.title('최적화 함수　비교 결과(검증 데이터의 정확도)')
plt.xlabel('반복 횟수')
plt.ylabel('정확도')
plt.legend()
plt.show()

## 10.8 드랍 아웃

In [ ]:
class CNN_v3(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # ... (이전 conv, relu, maxpool 등 정의는 동일) ...
        self.conv1 = nn.Conv2d(3, 32, 3, padding=(1,1))
        self.conv2 = nn.Conv2d(32, 32, 3, padding=(1,1))
        self.conv3 = nn.Conv2d(32, 64, 3, padding=(1,1))
        self.conv4 = nn.Conv2d(64, 64, 3, padding=(1,1))
        self.conv5 = nn.Conv2d(64, 128, 3, padding=(1,1))
        self.conv6 = nn.Conv2d(128, 128, 3, padding=(1,1))
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d((2,2))
        self.flatten = nn.Flatten()

        self.l1 = nn.Linear(4*4*128, 128)
        self.l2 = nn.Linear(128, 10)

        # 드롭아웃 레이어를 비율을 다르게 하여 3개 정의
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.4)

        self.features = nn.Sequential(
            self.conv1, self.relu,
            self.conv2, self.relu, self.maxpool,
            self.dropout1, # 첫 번째 MaxPool 뒤에 추가
            self.conv3, self.relu,
            self.conv4, self.relu, self.maxpool,
            self.dropout2, # 두 번째 MaxPool 뒤에 추가
            self.conv5, self.relu,
            self.conv6, self.relu, self.maxpool,
            self.dropout3, # 세 번째 MaxPool 뒤에 추가

        )
        self.classifier = nn.Sequential(
            self.l1, self.relu,
            self.dropout3, # 분류기의 선형 레이어 사이에 추가
            self.l2
        )

    def forward(self, x):
        x1 = self.features(x)
        x2 = self.flatten(x1)
        x3 = self.classifier(x2)
        return x3

In [ ]:
# 손실 계산 그래프 시각화
net = CNN_v3(n_output).to(device)
criterion = nn.CrossEntropyLoss()
loss = eval_loss(test_loader, device, net, criterion)
g = make_dot(loss, params=dict(net.named_parameters()))
display(g)

In [ ]:
# 난수 고정
torch_seed()

# 모델 인스턴스 생성
net = CNN_v3(n_output).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())
history = np.zeros((0, 5))

In [ ]:
# 학습

num_epochs = 50
history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history)

In [ ]:
evaluate_history(history)

## 10.9 배치 정규화

In [ ]:
class CNN_v4(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # ... (이전 conv, relu, maxpool, dropout 등 정의는 동일) ...
        self.conv1 = nn.Conv2d(3, 32, 3, padding=(1,1))
        self.conv2 = nn.Conv2d(32, 32, 3, padding=(1,1))
        self.conv3 = nn.Conv2d(32, 64, 3, padding=(1,1))
        self.conv4 = nn.Conv2d(64, 64, 3, padding=(1,1))
        self.conv5 = nn.Conv2d(64, 128, 3, padding=(1,1))
        self.conv6 = nn.Conv2d(128, 128, 3, padding=(1,1))
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d((2,2))
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.4)
        self.flatten = nn.Flatten()

        self.l1 = nn.Linear(128 * 4* 4, 512)
        self.l2 = nn.Linear(512, num_classes)


        # 각 Conv 레이어의 출력 채널 수에 맞춰 BatchNorm2d 레이어 정의
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm2d(64)
        self.bn5 = nn.BatchNorm2d(128)
        self.bn6 = nn.BatchNorm2d(128)

        self.features = nn.Sequential(
            self.conv1, self.bn1, self.relu, # conv1 -> bn1 -> relu
            self.conv2, self.bn2, self.relu, self.maxpool, # conv2 -> bn2 -> relu
            self.dropout1,
            self.conv3, self.bn3, self.relu, # conv3 -> bn3 -> relu
            self.conv4, self.bn4, self.relu, self.maxpool, # conv4 -> bn4 -> relu
            self.dropout2,
            self.conv5, self.bn5, self.relu, # conv5 -> bn5 -> relu
            self.conv6, self.bn6, self.relu, self.maxpool, # conv6 -> bn6 -> relu
            self.dropout3,
        )

        self.classifier = nn.Sequential(
            self.l1,
            self.relu,
            self.dropout3,
            self.l2
        )

    def forward(self, x):
        x1 = self.features(x)
        x2 = self.flatten(x1)
        x3 = self.classifier(x2)
        return x3

In [ ]:
# 난수 고정
torch_seed()

# 모델 인스턴스 생성
net = CNN_v4(n_output).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())
history = np.zeros((0, 5))

In [ ]:
# 학습

num_epochs = 50
history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history)

In [ ]:
evaluate_history(history)

## 10.10 데이터 증강 기법

In [ ]:
# 훈련 데이터용: 정규화에 반전과 RandomErasing 추가
transform_train = transforms.Compose([
  transforms.RandomHorizontalFlip(p=0.5), # 50% 확률로 좌우 반전
  transforms.ToTensor(),
  transforms.Normalize(0.5, 0.5),
  transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)
])

In [ ]:
# transfrom_train을 사용한 데이터셋 정의
train_set2 = datasets.CIFAR10(
    root = data_root, train = True,
    download = True, transform = transform_train)

# traisform_train을 사용한 데이터로더 정의
batch_size = 100
train_loader2 = DataLoader(train_set2, batch_size=batch_size, shuffle=True)

In [ ]:
# 새로운 훈련 데이터의 처음 50개 표시

# 난수 고정
torch_seed()

show_images_labels(train_loader2, classes, None, None)

In [ ]:
# 난수 고정
torch_seed()

# 모델 인스턴스 생성
net = CNN_v4(n_output).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())
history = np.zeros((0, 5))

In [ ]:
# 학습
# 동일한 모델에서 train_loader2로 데이터를 변경

num_epochs = 100
history = fit(net, optimizer, criterion, num_epochs,
        train_loader2, test_loader, device, history)

In [ ]:
evaluate_history(history)

In [ ]:
show_images_labels(test_loader, classes, net, device)

In [ ]:
# 잘못 예측한 38번째 데이터 추출
for images, labels in test_loader:
    break
image = images[37]
label = labels[37]

# 이미지 확인
plt.figure(figsize=(3,3))
w = image.numpy().copy()
# print(w)
print()
w2 = np.transpose(w, (1, 2, 0))
# print(w2)
print()
w3 = (w2 + 1)/2   # 역정규화
#print(w3)
print()
plt.title(classes[label])
plt.imshow(w3)
plt.show()

In [ ]:
# 예측 값 출력
image = image.view(1, 3, 32, 32) # 1 : 배치차원 추가
image = image.to(device)
output = net(image)

# 라벨 별 확률 값 출력
probs = torch.softmax(output, dim=1)
# logits(모델이 예측한 값) >> 확률로 변환(각 클래스에 속할 확률)
probs_np = probs.data.to('cpu').numpy()[0]
values = np.frompyfunc(lambda x: f'{x:.04f}', 1, 1)(probs_np)
names = np.array(classes)
tbl = np.array([names, values]).T
print(tbl)

## 칼럼 배치 정규화를 사용할 때 주의할 점

### 잘못된 모델 클래스 정의의 예시

In [ ]:
# # 잘못 정의된 모델 클래스
# class CNN_v5_wrong(nn.Module):
#     def __init__(self, num_classes=10):
#         super().__init__()
#         # ... (conv 레이어들은 이전과 같이 정의) ...
#         self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
#         self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
#         self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
#         self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
#         self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
#         self.conv6 = nn.Conv2d(128, 128, 3, padding=1)
#         self.relu = nn.ReLU(inplace=True)
#         self.maxpool = nn.MaxPool2d(2, 2)
#         self.dropout1 = nn.Dropout(0.2)
#         self.dropout2 = nn.Dropout(0.3)
#         self.dropout3 = nn.Dropout(0.4)

#         self.l1 = nn.Linear(4*4*128, 128)
#         self.l2 = nn.Linear(128, 10)

#         # BN 인스턴스를 3개만 생성
#         self.bn1 = nn.BatchNorm2d(32)
#         self.bn2 = nn.BatchNorm2d(64)
#         self.bn3 = nn.BatchNorm2d(128)

#         self.features = nn.Sequential(
#             self.conv1, self.bn1, self.relu,
#             self.conv2, self.bn1, self.relu, # 여기서 self.bn1을 재사용
#             self.maxpool, self.dropout1,
#             self.conv3, self.bn2, self.relu,
#             self.conv4, self.bn2, self.relu, # 여기서 self.bn2을 재사용
#             self.maxpool, self.dropout2,
#             self.conv5, self.bn3, self.relu,
#             self.conv6, self.bn3, self.relu, # 여기서 self.bn3을 재사용
#             self.maxpool, self.dropout3,
#         )

#         self.classifier = nn.Sequential(
#             self.l1,
#             self.relu,
#             self.dropout3,
#             self.l2
#         )

#     def forward(self, x):
#         x1 = self.features(x)
#         x2 = self.flatten(x1)
#         x3 = self.classifier(x2)
#         return x3

In [ ]:
# 잘못 정의된 모델 클래스
class CNN_v5(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # ... (conv 레이어들은 이전과 같이 정의) ...
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, 3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.4)
        self.flatten = nn.Flatten()

        self.l1 = nn.Linear(4*4*128, 128)
        self.l2 = nn.Linear(128, 10)

        # BN 인스턴스를 3개만 생성
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)

        self.features = nn.Sequential(
            self.conv1, self.bn1, self.relu,
            self.conv2, self.bn1, self.relu, # 여기서 self.bn1을 재사용
            self.maxpool, self.dropout1,
            self.conv3, self.bn2, self.relu,
            self.conv4, self.bn2, self.relu, # 여기서 self.bn2을 재사용
            self.maxpool, self.dropout2,
            self.conv5, self.bn3, self.relu,
            self.conv6, self.bn3, self.relu, # 여기서 self.bn3을 재사용
            self.maxpool, self.dropout3,
        )

        self.classifier = nn.Sequential(
            self.l1,
            self.relu,
            self.dropout3,
            self.l2
        )

    def forward(self, x):
        x1 = self.features(x)
        x2 = self.flatten(x1)
        x3 = self.classifier(x2)
        return x3

In [ ]:
# 난수 고정
torch_seed()

# 모델 인스턴스 생성
net = CNN_v5(n_output).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())
history = np.zeros((0, 5))

In [ ]:
# 학습

num_epochs = 50
history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history)

In [ ]:
# 손실 계산 그래프 시각화
net = CNN_v5(n_output).to(device)
criterion = nn.CrossEntropyLoss()
loss = eval_loss(test_loader, device, net, criterion)
g = make_dot(loss, params=dict(net.named_parameters()))
display(g)

### 칼럼 배치 정규화에서 처리하는 내용
### 배치 정규화 동작 원리 코드

In [ ]:
# 입력용 더미 데이터 작성
torch.manual_seed(123)
inputs = torch.randn(1, 1, 10)
print(inputs)

In [ ]:
# 입력 미니 배치 데이터의 통계량 산출
i_mean = inputs.mean()
i_var = inputs.var(unbiased=True) # running_var 업데이트에 사용
i_std = inputs.std(unbiased=False) # 훈련 출력 계산에 사용
print(f"입력 평균: {i_mean:.4f}, 입력 표준편차: {i_std:.4f}, 입력 분산: {i_var:.4f}\n")
# unbiased variance 불편(편향 bias 이 없음) 분산 (표본 분산 사용)
# unbiased=True >> 분모가 n-1 (표본분산)
# unbiased=False >> 분모가 n (모집단 표준편차)

In [ ]:
# BN 함수 정의 및 초기 상태 출력
bn = nn.BatchNorm1d(1) # 1차원 배치 정규화 (채널수 1)
print(f"초기 running_mean: {bn.running_mean.data}")
print(f"초기 running_var: {bn.running_var.data}")
print(f"초기 weight: {bn.weight.data}")
print(f"초기 bias: {bn.bias.data}")

In [ ]:
# BN 함수의 유사 호출
bn.train()
outputs1_train = bn(inputs)
print(f"--- 훈련 페이즈 1 ---")
print(f"출력: {outputs1_train.data.numpy().round(4)}")
print(f"이후 running_mean: {bn.running_mean.data.numpy().round(4)}")
print(f"이후 running_var: {bn.running_var.data.numpy().round(4)}\n")

bn.eval()
outputs1_eval = bn(inputs)
print(f"--- 예측 페이지 1 ---")
print(f"출력: {outputs1_eval.data.numpy().round(4)}")
print(f"이후 running_mean: {bn.running_mean.data.numpy().round(4)}")
print(f"이후 running_var: {bn.running_var.data.numpy().round(4)}\n")

bn.train()
outputs2_train = bn(inputs)
print(f"--- 훈련 페이즈 2 ---")
print(f"출력: {outputs2_train.data.numpy().round(4)}")
print(f"이후 running_mean: {bn.running_mean.data.numpy().round(4)}")
print(f"이후 running_var: {bn.running_var.data.numpy().round(4)}\n")

bn.eval()
outputs2_eval = bn(inputs)
print(f"--- 예측 페이지 2 ---")
print(f"출력: {outputs2_eval.data.numpy().round(4)}")
print(f"이후 running_mean: {bn.running_mean.data.numpy().round(4)}")
print(f"이후 running_var: {bn.running_var.data.numpy().round(4)}\n")

In [ ]:
# 훈련 페이즈 계산 검증
xt = (inputs - i_mean)/i_std * bn.weight + bn.bias
# 표준정규분포(표준화) 평균 0, 표준편차 1
# 공식: x - x_bar(mean) / sigma(표준편차)
# 이 표준화 공식에 gamma( bn.weight) 스케일링 해주고, beta (배치정규화한 편향) 더함

print("수동 훈련 출력과 파이토치 결과 일치:", torch.allclose(xt.data, outputs2_train.data))
print(xt.data)
print(outputs2_train.data)

# torch.allclose : 제시된 두 개의 텐서가 같거나 유사한지 확인(부동소수점 오차까지만 허용)

In [ ]:
# 예측 페이즈 계산 검증
xp = (inputs-bn.running_mean)/torch.sqrt(bn.running_var)

print("수동 예측 출력과 파이토치 결과 일치:", torch.allclose(xp.data, outputs2_eval.data))
print(xp.data)
print(outputs2_eval.data)

In [ ]:
# running_mean과 runnung_var의 계산식

# 초깃값
mean0 = 0
var0 = 1
momentum = bn.momentum

# 이동 평균 계산 1회차 -> bn.running_mean 은 2회차까지 갱신된 값이라 비교 시 False
mean1 = (1-momentum) * mean0 +  momentum * i_mean
var1 = (1-momentum) * var0 +  momentum * i_var
print("수동 running_mean과 파이토치 결과 일치:", torch.allclose(mean1, bn.running_mean))
print(mean1, var1)

# 이동 평균 계산 2회차 -> 2회차 기준 값이기 때문에 비교 시 True
mean2 = (1-momentum) * mean1 +  momentum * i_mean
var2 = (1-momentum) * var1 +  momentum * i_var
print("수동 running_var과 파이토치 결과 일치:", torch.allclose(var2, bn.running_var))
print(mean2, var2)